In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import random
from scipy import stats
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [2]:
# assign directory
ROOT_DIR = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
data_dir = ROOT_DIR + 'Data/Panoptic Agriculture/Cleaned Dataset'
 
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [3]:
df_dict = dict()
image = Image.open(file_list[0]).convert('L')
first_image = pywt.wavedec2(image, 'db1')
layer_len = len(first_image)
print(str(layer_len) + " layers being used")
for i in range(layer_len):
        df = pd.DataFrame(columns=["Image ID", "Orientation", "Data"])
        df_dict[i+1] = df

8 layers being used


In [4]:
for k in range(len(file_list)):
    image = Image.open(file_list[k]).convert('L')
    image = np.array(image)
   
    name = file_names[k].split(".")[0]
    transformed = pywt.wavedec2(image, 'db1')
    df_dict[1].loc[len(df_dict[1].index)] = [name, "L1", np.array(transformed[0][0]).flatten()]
    direction_names = ['H', 'V', 'D']

    for i in range(1, layer_len): 
        for j in range(len(transformed[i])):
            arr = np.array(transformed[i][j])
            df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr.flatten()]


In [5]:
def convert_to_wavelet_basis(folder_dir,  normalized = True, basis="db1"):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    file_names = os.listdir(folder_dir)
    #Setup df Dict
    df_dict = dict()
    image = Image.open(file_list[0]).convert('L')
    first_image = pywt.wavedec2(image, basis)
    layer_len = len(first_image)
    print(str(layer_len) + " layers being used")
    for i in range(layer_len):
        df = pd.DataFrame(columns=["Image ID", "Orientation", "Data"])
        df_dict[i+1] = df
    
    
    #Fill DF DICT
    for k in range(len(file_list)):
        image = Image.open(file_list[k]).convert('L')
        image = np.array(image)
        if normalized:
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std 
            
        name = file_names[k].split(".")[0]
        transformed = pywt.wavedec2(image, basis)
        df_dict[1].loc[len(df_dict[1].index)] = [name, "L1", np.array(transformed[0][0]).flatten()]
        direction_names = ['H', 'V', 'D']

        for i in range(1, layer_len): 
            for j in range(len(transformed[i])):
                arr = np.array(transformed[i][j])
                df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr.flatten()]

    return df_dict

In [6]:
converted = convert_to_wavelet_basis(data_dir, normalized = False)

8 layers being used


In [7]:
converted[2]

,Image ID,Orientation,Data
0,test900,H,[2441.0703125000023]
1,test900,V,[991.3515625000009]
2,test900,D,[1248.6328125000014]
3,test1629,H,[-310.8281249999998]
4,test1629,V,[-326.2343750000009]
...,...,...,...
4765,test1156,V,[-166.89843750000045]
4766,test1156,D,[359.1484375000007]
4767,test2339,H,[2388.1640625000027]
4768,test2339,V,[1628.7734375000032]


In [8]:
def dict_to_pickle(converted_directory, converted, name):
    filename = name
    filename = os.path.join(converted_directory, filename)
    with open(filename+".pickle", 'wb') as handle:
        pickle.dump(converted, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
norm_converted_directory = ROOT_DIR + 'data/Panoptic Agriculture/Transformed Dataset/'
norm_converted = convert_to_wavelet_basis(data_dir, normalized = True)


8 layers being used


In [11]:
norm_converted_directory

'/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/data/Panoptic Agriculture/Transformed Dataset/'

In [12]:
dict_to_pickle(norm_converted_directory, norm_converted, "Panoptic_Data_Dict_Normalized")